In [1]:
NODE_TYPE=compute_haswell
LEASE_NAME="$USER-tutorial-$(date +%b%d)"
KEYPAIR_NAME="$USER-jupyter-$(hostname)"
SERVER_NAME="$LEASE_NAME"
# FLOATING_IP will be set later; it is dynamically reserved

## Step 1: Authenticate with your Chameleon project

In [2]:
if [[ "${OS_PROJECT_NAME:+x}" != "x" ]]; then
  echo "No project could automatically be detected. Overriding in Notebook."
  # For example...
  export OS_PROJECT_NAME='CH-000000'
fi

# Test that you can authenticate
openstack token issue >/dev/null && echo "Successfully authenticated to project $OS_PROJECT_NAME"

Successfully authenticated to project CH-821000


## Step 2: Create a reservation

We're reserving a bare metal node and a public IP address on Chameleon. While Chameleon has several nodes and IP addresses for use by researchers, the supply is still constrained relative to the amount of people using the system, so, unlike commercial clouds like AWS, it is important to reserve them ahead of time.

In [3]:
PUBLIC_NETWORK_ID=$(openstack network show public -f value -c id)

blazar lease-create \
  --physical-reservation min=1,max=1,resource_properties="[\"=\", \"\$node_type\", \"$NODE_TYPE\"]" \
  --reservation "resource_type=virtual:floatingip,network_id=$PUBLIC_NETWORK_ID,amount=1" \
  --start-date "$(date +'%Y-%m-%d %H:%M')" \
  --end-date "$(date +'%Y-%m-%d %H:%M' -d'+1 day')" \
  "$LEASE_NAME"

Matches: ('min=1,max=1', 'resource_properties', '["=", "$node_type", "compute_haswell"]')
Matches: ('min=1', 'max', '1')
Matches: (None, 'min', '1')
Matches: ('resource_type=virtual:floatingip,network_id=44b38c44-2a42-4b6d-b129-6c8f1b2a1375', 'amount', '1')
Matches: ('resource_type=virtual:floatingip', 'network_id', '44b38c44-2a42-4b6d-b129-6c8f1b2a1375')
Matches: (None, 'resource_type', 'virtual:floatingip')
Created a new lease:
+--------------+----------------------------------------------------------------------------+
| Field        | Value                                                                      |
+--------------+----------------------------------------------------------------------------+
| created_at   | 2019-08-13 18:29:53                                                        |
| degraded     | False                                                                      |
| end_date     | 2019-08-14T18:29:00.000000                                                 |
| 

In [4]:
wait_lease "$LEASE_NAME"

Waiting up to 300 seconds for lease abhinav5-tutorial-Aug13 to start...
Lease started successfully!


## Step 3: Provision a new bare metal instance

Now let's use that reserved node and launch a bare metal instance on it. We will start by launching existing images maintained by Chameleon.

### Create an SSH keypair

In [6]:
openstack keypair create --public-key ~/work/.ssh/id_rsa.pub "$KEYPAIR_NAME"

+-------------+-------------------------------------------------+
| Field       | Value                                           |
+-------------+-------------------------------------------------+
| fingerprint | 25:be:13:4e:80:bf:d6:a9:04:a6:17:fc:f6:fc:f3:a4 |
| name        | abhinav5-jupyter-a471b14a894c                   |
| user_id     | bbc7610b4b30405bb5a0c2265ff0ca3b                |
+-------------+-------------------------------------------------+


### Launch an instance

In [7]:
SERVER_NAME="$LEASE_NAME"

openstack server create --key-name "$KEYPAIR_NAME" \
  $(lease_server_create_default_args "$LEASE_NAME") \
  $SERVER_NAME

+-------------------------------------+---------------------------------------------------+
| Field                               | Value                                             |
+-------------------------------------+---------------------------------------------------+
| OS-DCF:diskConfig                   | MANUAL                                            |
| OS-EXT-AZ:availability_zone         |                                                   |
| OS-EXT-SRV-ATTR:host                | None                                              |
| OS-EXT-SRV-ATTR:hypervisor_hostname | None                                              |
| OS-EXT-SRV-ATTR:instance_name       |                                                   |
| OS-EXT-STS:power_state              | NOSTATE                                           |
| OS-EXT-STS:task_state               | scheduling                                        |
| OS-EXT-STS:vm_state                 | building                                

In [8]:
wait_instance "$SERVER_NAME"

Waiting up to 600 seconds for instance abhinav5-tutorial-Aug13 to start
Instance created successfully!


## Step 4: Assign a public IP address

By default, your new instance is assigned only an internal private IP, and is not reachable from the public internet. In order to reach your node, you must attach a public IP (called a "Floating IP", because it can "float" between instances over time; they are shared resources.)

We will attach the public IP that was reserved for us. `lease_list_floating_ips` helper function will print a list of public IPs reserved as part of your lease.

In [9]:
FLOATING_IP=$(lease_list_floating_ips "$LEASE_NAME" | head -n1)

openstack server add floating ip "$SERVER_NAME" "$FLOATING_IP" \
  && echo "Attached floating ip $FLOATING_IP!"

Attached floating ip 192.5.87.223!


In [10]:
wait_ssh "$FLOATING_IP"

Waiting up to 300 seconds for SSH on 192.5.87.223...
SSH is running!


## Step 5: Install the service

### Start an SSH tunnel to the instance

In [13]:
# ssh -L 8080:localhost:8080 cc@"$FLOATING_IP" echo 'User $(whoami) connected on $(hostname)!'
ssh cc@"$FLOATING_IP"

cc@192.5.87.223: Permission denied (publickey,gssapi-keyex,gssapi-with-mic).


: 255

### Install

In [ ]:
sudo yum install git
git clone https://github.com/dbalouek/ChoreoMicroservices.git
cd ChoreoMicroservices/ 
git checkout express
chmod +x install.sh
./install.sh

## Step 5: Clean up (optional)

If you don't need your server anymore, it is a nice thing to end your lease yourself, rather than waiting for it to expire. Ending a lease immediately returns resources to be reserved by others. You don't need to stop your servers; that will be done automatically when the lease is deleted.

In [14]:
# Commented out for safety.
# blazar lease-delete "$LEASE_NAME"